In [2]:
from PIL import Image
import requests
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel

results_df = pd.DataFrame(columns=['image_url', 'image_embedding', 'parse_file'])
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

new_df = pd.DataFrame(columns=['image_url', 'image_embedding', 'parse_file'])
with open(datafile_name + "_embeddings.json", "w") as datafile:
    for index, row in originaldf.head(50).iterrows():
        url = row['image_url']
        hash = row['parse_file']
        try:
            response = requests.get(url, timeout = 5)
            content_type = response.headers.get('content-type')
            image = Image.open(BytesIO(response.content))
        
            inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)
        
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
            probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
            embeddings = outputs.image_embeds[0].detach().numpy().tolist()
            
            new_row = {
                'image_url': url,
                'image_embedding':embeddings,
                'parse_file': hash
            }
            
            datafile.write(json.dumps(new_row) + '\n')
            print("Successful dump #" + str(index + 1) + '\n')
        except Exception as e:
            print(f"Skipping row with URL: {url}")
            print(e)

        new_df = new_df.append(new_row, ignore_index=True)
        print("Successful" + '\n')
# with open(datafile_name + "_embeddings.json", "rt") as datafile:
#     ldf = pd.read_json(datafile, lines=True)
# display(ldf) #uncomment for entire dataset
display(new_df)
    # results_df = results_df.append(new_row, ignore_index=True)
    # results_df.to_json(datafile_name + "_embeddings.json.gz", orient='records', compression = 'gzip')


Hello world


In [ ]:
#This block will hold all imports and internal functions
from PIL import Image
import requests
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel
import json
import gzip
import pandas as pd

def _extractURLFromFile(input_urls):
    if isinstance(input_urls, str):  # Check if input_urls is a single file path
        input_urls = [input_urls]  # Convert it to a list with a single element

    urls = []
    for file_path in input_urls:
    # Detect file type based on extension
        file_extension = file_path.rsplit('.', 1)[-1].lower()

        if file_extension == 'json':
            with open(input_urls, 'r') as datafile:
                data = json.load(datafile)
                _extractURLFromJSON(data, urls)
    else:
        with open(input_urls, 'r') as datafile:
            for line in datafile:
                url = line.strip()
                if url:  # Skip empty lines
                    urls.append(url)
    return urls

def _extractURLFromJSON(data, urls):
    if isinstance(data, list):
        for item in data:
            _extractURLFromJSON(item, urls)
    elif isinstance(data, dict):
        for value in data.values():
            if isinstance(value, str) and value.startswith('http'):
                urls.append(value)
            elif isinstance(value, (dict, list)):
                _extractURLFromJSON(value, urls)

def _imageEmbeddings(url_list)
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")  #Loading pre-trained CLIP model/processor
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    url_embeddings_list = []
    for url in url_list
        try:
            response = requests.get(url, timeout = 5)  #For URLs that are unretrievable. 5 secs for slow retrieval
            content_type = response.headers.get('content-type')
            image = Image.open(BytesIO(response.content))
            
            inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True) #Inputs for the model; Can't change text inputs or get an exception
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
            probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
            embeddings = outputs.image_embeds[0].detach().numpy().tolist()
            url_embeddings_list.append((url, embeddings))

        except Exception as e:
            print(f"Skipping URL: {url}")
            print(e)
    return url_embeddings_list


In [ ]:
#User-callable function to create image embeddings
def createImageEmbeddings(input_urls): # ONLY RETURNS LIST FOR THE TIME BEING 
    if isinstance(input_urls, str):
        # Read the file and process its contents
        url_list = _extractURLFromFile(input_urls)
        embeddings = _imageEmbeddings(url_list)
                             
    if isinstance(input_urls, list):
        embeddings = []
        # Multiple URLs or file names provided
        for url in input_urls:
            if isinstance(url, str):
                # Process a file
                url_list = _extractURLFromFile(url)
                embeddings.extend(_imageEmbeddings(url_list))
            elif isinstance(url, str) and url.startswith('http'):
                # Process a URL
                embeddings.extend(_imageEmbeddings(url))
        return embeddings
            else:
                raise ValueError("Invalid item in the input list. Expected file name (string) or URL (string).")
    else:
        raise ValueError("Invalid input type. Expected list or file name (string).")

In [ ]:
#User-callable function to generate text embeddings
def createTextEmbeddings(inputText)
    